In [5]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.0.2p             |       h470a237_1         3.1 MB  conda-forge
    certifi-2018.10.15         |        py36_1000         138 KB  conda-forge
    geopy-1.17.0               |             py_0          49 KB  conda-forge
    ca-certificates-2018.10.15 |       ha4d7672_0         135 KB  conda-forge
    conda-4.5.11               |        py36_1000         651 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.1 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0            conda-forge
    geopy:           

# Data sets

The data that will be used in this solution is from a json file available at <a href='https://ibm.box.com/shared/static/fbpwbovar7lf8p5sgddm06cgipa2rxpe.json'>https://ibm.box.com/shared/static/fbpwbovar7lf8p5sgddm06cgipa2rxpe.json</a>. The json file has the data related to all neighborhoods in the city of NewYork.

In [6]:
!wget -q -O 'newyork_data.json' https://ibm.box.com/shared/static/fbpwbovar7lf8p5sgddm06cgipa2rxpe.json
print('Data downloaded!')

Data downloaded!


In [7]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [8]:
neighborhoods_data = newyork_data['features']

Creating dataframes with the neighborhood data of Newyork city.

In [9]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [10]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In this solution we will focus on the Neighborhoods in Manhattan and Brooklyn. So, we will create separate dataframes containing the neighborhoods of Manhattan and Brooklyn 

In [11]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
brooklyn_data = neighborhoods[neighborhoods['Borough'] == 'Brooklyn'].reset_index(drop=True)

## Visualizing the neighborhoods of Manhattan on a map

In [12]:
address = 'Manhattan, NY'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Manhattan are 40.7900869, -73.9598295.


In [13]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

## Visualizing the neighborhoods of Brooklyn

In [15]:
brooklyn_address = 'Brooklyn, NY'

geolocator = Nominatim()
location = geolocator.geocode(brooklyn_address)
brooklyn_latitude = location.latitude
brooklyn_longitude = location.longitude
print('The geograpical coordinate of Brroklyn are {}, {}.'.format(brooklyn_latitude, brooklyn_longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Brroklyn are 40.6501038, -73.9495823.


In [16]:
# create map of Brooklyn using latitude and longitude values
map_brooklyn = folium.Map(location=[brooklyn_latitude, brooklyn_longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(brooklyn_data['Latitude'], brooklyn_data['Longitude'], brooklyn_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_brooklyn)  
    
map_brooklyn

## Explore the neighborhoods of Manhattan using Foursquare API

In [18]:
CLIENT_ID = 'G5MJSGEHGMC3PIAZT3GOI2AZ4JBJBN3PSV1R0HWB0RTLXJ3Q' # your Foursquare ID
CLIENT_SECRET = '0JCK4QBOBMLSEF1ESBHLFCNOKHM0NLQSTLUGQACASBPE4KRE' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT=100


In [19]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        #print(url)   
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [22]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )


In [25]:
print('There are {} venues in Manhattan.'.format(manhattan_venues.shape[0]))

There are 3294 venues in Manhattan.


## Explore the neighborhoods of Brooklyn using Foursquare API

In [26]:
brooklyn_venues = getNearbyVenues(names=brooklyn_data['Neighborhood'],
                                   latitudes=brooklyn_data['Latitude'],
                                   longitudes=brooklyn_data['Longitude']
                                  )

In [27]:
print('There are {} venues in Brooklyn.'.format(brooklyn_venues.shape[0]))

There are 2816 venues in Brooklyn.


## Filter the avenues in both Manhattan and Brooklyn to get the restaurants data

In [28]:
def asianrestaurantcount(row):
    neighborhood=row['Neighborhood']
    restaurants_neighborhood=restaurants_df[restaurants_df['Neighborhood']==neighborhood]
    asianrestaurants_neighborhood=restaurants_neighborhood[restaurants_neighborhood['Venue Category']=='Asian Restaurant']
    return asianrestaurants_neighborhood.shape[0]

In [33]:
def brooklyn_asianrestaurantcount(row):
    neighborhood=row['Neighborhood']
    brooklyn_restaurants_neighborhood=brooklyn_restaurants_df[brooklyn_restaurants_df['Neighborhood']==neighborhood]
    brooklyn_asianrestaurants_neighborhood=brooklyn_restaurants_neighborhood[brooklyn_restaurants_neighborhood['Venue Category']=='Asian Restaurant']
    return brooklyn_asianrestaurants_neighborhood.shape[0]

In [29]:
restaurants_df=manhattan_venues[manhattan_venues['Venue Category'].str.contains('Restaurant')].reset_index(drop=True)
restaurants=restaurants_df.groupby(['Neighborhood']).size().reset_index(name='Restaurant count')
print('Total number of restarunats in Manhattan is {}'.format(restaurants['Restaurant count'].sum()))


Total number of restarunats in Manhattan is 948


In [30]:
brooklyn_restaurants_df=brooklyn_venues[brooklyn_venues['Venue Category'].str.contains('Restaurant')].reset_index(drop=True)
brooklyn_restaurants=brooklyn_restaurants_df.groupby(['Neighborhood']).size().reset_index(name='Restaurant count')
print('Total number of restarunats in Brooklyn is {}'.format(brooklyn_restaurants['Restaurant count'].sum()))

Total number of restarunats in Brooklyn is 672


Manhattan has more number of restaurants than Brooklyn. Now, lets further filter the restaurants to find the number of Asian Restaurants.

In [31]:
restaurants['Asian Restaurant count']=restaurants.apply(asianrestaurantcount,axis=1)
print('Total number of Asian restarunats in Manhattan is {}'.format(restaurants['Asian Restaurant count'].sum()))

Total number of Asian restarunats in Manhattan is 14


In [34]:
brooklyn_restaurants['Asian Restaurant count']=brooklyn_restaurants.apply(brooklyn_asianrestaurantcount,axis=1)
print('Total number of Asian restarunats in Brooklyn is {}'.format(brooklyn_restaurants['Asian Restaurant count'].sum()))

Total number of Asian restarunats in Brooklyn is 16


#### Though Manhattan has more restaurants than Brooklyn, it has fewer Asian restaurants than Brooklyn. So, Manhattan appears to be better option to open a Asian restaurant

## Lets dig in a little deeper to see the count of restaurants and Asian restaurants in each of the neighborhoods of Manhattan.

Count of restaurants and Asian restaurants in Manhattan grouped by Neighborhood

In [35]:
restaurants.sort_values(by='Restaurant count',ascending=False)

,Neighborhood,Restaurant count,Asian Restaurant count
12,Greenwich Village,46,0
4,Chinatown,42,2
34,Turtle Bay,38,1
38,West Village,37,0
36,Upper West Side,37,1
8,East Village,37,0
26,Murray Hill,35,0
27,Noho,33,1
10,Flatiron,33,0
31,Sutton Place,31,0


From the above summary, Greenwich Village and Chinatown have the most number of restaurant among the neighborhoods. In general, more number of restaurants implies that there is good demand for restaurants in that locality. Lets further study the neighborhoods of Greenwich Village and Chinatown to zero down on the best location for an Asian Restaurant

In [36]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")
print(manhattan_onehot.shape)
# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 
print(manhattan_onehot.shape)
# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]
manhattan_onehot.head()

(3294, 328)
(3294, 329)


,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundromat,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Snack Place,Soba Restaurant,Social Club,Soup Place,South Indian Restaurant,Southern / Soul Food Restaurant,

In [38]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()


#### Let's study each neighborhood in Manhattan to know the top 5 most common venues in that neighborhood.

In [39]:
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
                venue  freq
0         Coffee Shop  0.09
1                Park  0.07
2               Hotel  0.05
3  Italian Restaurant  0.03
4                 Gym  0.03


----Carnegie Hill----
                venue  freq
0         Pizza Place  0.06
1      Cosmetics Shop  0.05
2         Coffee Shop  0.05
3                Café  0.04
4  Italian Restaurant  0.03


----Central Harlem----
                 venue  freq
0   African Restaurant  0.07
1   Chinese Restaurant  0.05
2  American Restaurant  0.05
3   Seafood Restaurant  0.05
4       Cosmetics Shop  0.05


----Chelsea----
                venue  freq
0  Italian Restaurant  0.06
1         Coffee Shop  0.06
2      Ice Cream Shop  0.04
3              Bakery  0.04
4           Nightclub  0.04


----Chinatown----
                   venue  freq
0     Chinese Restaurant  0.09
1  Vietnamese Restaurant  0.04
2    American Restaurant  0.04
3        Bubble Tea Shop  0.04
4     Dim Sum Restaurant  0.04


----Civic Center----


We can see that both Greenwich Village and Chinatown have restaurants as the 4 out of 5 most common venues in the neighborhood. But, the restaurants in Chinatown appear to be those serving cuisines which overlap with Asian cuisine. Whereas, the restaurants in Greenwich Village as more spread out in terms of the cuisine they offer. 

Further, Chitatown already has 2 Asian restaurants whereas Greenwich Village doesn't have any. So, an Asian restaurant would be a good addition to the restaurants available in it.

Let's study further to see if we can find any reason to support our inference that Greenwich Village is a good location to open an Asian restaurant.

In [44]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [45]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Coffee Shop,Park,Hotel,Wine Shop,Italian Restaurant,Gym,Ice Cream Shop,Market,Fountain,Burger Joint
1,Carnegie Hill,Pizza Place,Coffee Shop,Cosmetics Shop,Café,Yoga Studio,Japanese Restaurant,Bookstore,Bar,French Restaurant,Spa
2,Central Harlem,African Restaurant,Seafood Restaurant,Cosmetics Shop,American Restaurant,French Restaurant,Chinese Restaurant,Public Art,Bookstore,Library,Cafeteria
3,Chelsea,Italian Restaurant,Coffee Shop,Bakery,Nightclub,Ice Cream Shop,American Restaurant,Seafood Restaurant,Theater,Hotel,Art Gallery
4,Chinatown,Chinese Restaurant,Dim Sum Restaurant,Bubble Tea Shop,American Restaurant,Vietnamese Restaurant,Bakery,Salon / Barbershop,Bar,Noodle House,Ice Cream Shop
5,Civic Center,Gym / Fitness Center,Cocktail Bar,Italian Restaurant,Bakery,Park,Sushi Restaurant,Sporting Goods Shop,Spa,French Restaurant,Sandwich Place
6,Clinton,Theater,Italian Restaurant,Hotel,American Restaurant,Gym / Fitness Center,Wine Shop,Gym,Indie Theater,Coffee Shop,French Restaurant
7,East Harlem,Mexican Restaurant,Bakery,Latin American Restaurant,Thai Restaurant,Deli / Bodega,Pizza Place,Spa,Beer Bar,Taco Place,French Restaurant
8,East Village,Ice Cream Shop,Bar,Wine Bar,Mexican Restaurant,Chinese Restaurant,Cocktail Bar,Pizza Place,Café,Italian Restaurant,Coffee Shop
9,Financial District,Coffee Shop,Hotel,Italian Restaurant,Steakhouse,Wine Shop,Food Truck,Juice Bar,Bar,Japanese Restaurant,Gym


In [46]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 1, 0, 0, 0, 0, 0, 4, 1, 3], dtype=int32)

In [47]:
manhattan_merged = manhattan_data

# add clustering labels
manhattan_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,3,Sandwich Place,Discount Store,Yoga Studio,Coffee Shop,Supplement Shop,Shoe Store,Tennis Stadium,Seafood Restaurant,Bank,Donut Shop
1,Manhattan,Chinatown,40.715618,-73.994279,1,Chinese Restaurant,Dim Sum Restaurant,Bubble Tea Shop,American Restaurant,Vietnamese Restaurant,Bakery,Salon / Barbershop,Bar,Noodle House,Ice Cream Shop
2,Manhattan,Washington Heights,40.851903,-73.936900,0,Café,Mobile Phone Shop,Sandwich Place,Bakery,Grocery Store,Tapas Restaurant,Supermarket,Pizza Place,Liquor Store,Lounge
3,Manhattan,Inwood,40.867684,-73.921210,0,Mexican Restaurant,Pizza Place,Bakery,Lounge,Café,Frozen Yogurt Shop,Pharmacy,Chinese Restaurant,Park,Restaurant
4,Manhattan,Hamilton Heights,40.823604,-73.949688,0,Mexican Restaurant,Café,Coffee Shop,Pizza Place,Yoga Studio,Sushi Restaurant,Caribbean Restaurant,Bakery,Chinese Restaurant,School


In [48]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [49]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Washington Heights,Café,Mobile Phone Shop,Sandwich Place,Bakery,Grocery Store,Tapas Restaurant,Supermarket,Pizza Place,Liquor Store,Lounge
3,Inwood,Mexican Restaurant,Pizza Place,Bakery,Lounge,Café,Frozen Yogurt Shop,Pharmacy,Chinese Restaurant,Park,Restaurant
4,Hamilton Heights,Mexican Restaurant,Café,Coffee Shop,Pizza Place,Yoga Studio,Sushi Restaurant,Caribbean Restaurant,Bakery,Chinese Restaurant,School
5,Manhattanville,Seafood Restaurant,Coffee Shop,Deli / Bodega,Chinese Restaurant,Italian Restaurant,Mexican Restaurant,Spanish Restaurant,Music School,Burger Joint,Spa
6,Central Harlem,African Restaurant,Seafood Restaurant,Cosmetics Shop,American Restaurant,French Restaurant,Chinese Restaurant,Public Art,Bookstore,Library,Cafeteria
10,Lenox Hill,Italian Restaurant,Coffee Shop,Sushi Restaurant,Pizza Place,Gym / Fitness Center,Burger Joint,Gym,Deli / Bodega,Café,Turkish Restaurant
12,Upper West Side,Italian Restaurant,Bar,Coffee Shop,Wine Bar,Indian Restaurant,Bakery,Burger Joint,Vegetarian / Vegan Restaurant,French Restaurant,Pub
17,Chelsea,Italian Restaurant,Coffee Shop,Bakery,Nightclub,Ice Cream Shop,American Restaurant,Seafood Restaurant,Theater,Hotel,Art Gallery
18,Greenwich Village,Italian Restaurant,Sushi Restaurant,Clothing Store,French Restaurant,Indian Restaurant,Bakery,American Restaurant,Chinese Restaurant,Café,Seafood Restaurant
21,Tribeca,American Restaurant,Italian Restaurant,Spa,Park,Gym,Wine Shop,Wine Bar,Cocktail Bar,Greek Restaurant,Bakery


### The neighborhoods in cluster 0  predominantly have eatery joints as the top 10 common venues. Greenwich Village is also in this cluster, this further strengths our inference that it is a good option to open an Asian restaurant.

### The added advantage for Greenwich village is it is in close proximity of affluent neighborhoods such as Soho and Tribeca. So, it is quite possible that people of these affluent neighborhoods would visit a restaurant that serves a new cuisine in a nearby locality.

## Based on the above findings, we can conclude that Greenwich village is the best neighborhood among the neighborhoods of Manhattan and Brooklyn to open an Asian Restaurant.

In [50]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Chinese Restaurant,Dim Sum Restaurant,Bubble Tea Shop,American Restaurant,Vietnamese Restaurant,Bakery,Salon / Barbershop,Bar,Noodle House,Ice Cream Shop
8,Upper East Side,Italian Restaurant,Exhibit,Juice Bar,Bakery,Art Gallery,Coffee Shop,Gym / Fitness Center,French Restaurant,Spa,Boutique
11,Roosevelt Island,Park,Sandwich Place,Monument / Landmark,Dog Run,Bus Line,Bus Station,Supermarket,Farmers Market,Metro Station,Outdoors & Recreation
16,Murray Hill,Japanese Restaurant,Hotel,Gym,Sandwich Place,Coffee Shop,Bar,Salon / Barbershop,Italian Restaurant,French Restaurant,Juice Bar
20,Lower East Side,Café,Coffee Shop,Bakery,Art Gallery,Shoe Store,Sandwich Place,Japanese Restaurant,Chinese Restaurant,Ramen Restaurant,Cocktail Bar
26,Morningside Heights,Coffee Shop,Park,Bookstore,American Restaurant,Pizza Place,Food Truck,Tennis Court,Sandwich Place,Burger Joint,Deli / Bodega
27,Gramercy,Italian Restaurant,Bagel Shop,Coffee Shop,Cocktail Bar,Vietnamese Restaurant,American Restaurant,Wine Shop,Mexican Restaurant,Hotel,Pizza Place
39,Hudson Yards,Coffee Shop,Hotel,Italian Restaurant,Café,American Restaurant,Theater,Gym / Fitness Center,Food Truck,Sandwich Place,Salad Place


In [52]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
30,Carnegie Hill,Pizza Place,Coffee Shop,Cosmetics Shop,Café,Yoga Studio,Japanese Restaurant,Bookstore,Bar,French Restaurant,Spa


In [53]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Sandwich Place,Discount Store,Yoga Studio,Coffee Shop,Supplement Shop,Shoe Store,Tennis Stadium,Seafood Restaurant,Bank,Donut Shop
9,Yorkville,Coffee Shop,Italian Restaurant,Bar,Gym,Sushi Restaurant,Pizza Place,Japanese Restaurant,Deli / Bodega,Ice Cream Shop,Pub
14,Clinton,Theater,Italian Restaurant,Hotel,American Restaurant,Gym / Fitness Center,Wine Shop,Gym,Indie Theater,Coffee Shop,French Restaurant
25,Manhattan Valley,Pizza Place,Coffee Shop,Bar,Yoga Studio,Mexican Restaurant,Indian Restaurant,Italian Restaurant,Café,American Restaurant,French Restaurant
28,Battery Park City,Coffee Shop,Park,Hotel,Wine Shop,Italian Restaurant,Gym,Ice Cream Shop,Market,Fountain,Burger Joint


In [54]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,East Harlem,Mexican Restaurant,Bakery,Latin American Restaurant,Thai Restaurant,Deli / Bodega,Pizza Place,Spa,Beer Bar,Taco Place,French Restaurant
13,Lincoln Square,Gym / Fitness Center,Theater,Plaza,Concert Hall,Italian Restaurant,French Restaurant,Indie Movie Theater,Bakery,Café,Opera House
15,Midtown,Hotel,Coffee Shop,Theater,Clothing Store,Steakhouse,Cocktail Bar,Sandwich Place,Bakery,Sporting Goods Shop,Park
19,East Village,Ice Cream Shop,Bar,Wine Bar,Mexican Restaurant,Chinese Restaurant,Cocktail Bar,Pizza Place,Café,Italian Restaurant,Coffee Shop
22,Little Italy,Bakery,Café,Yoga Studio,Cocktail Bar,Bubble Tea Shop,Seafood Restaurant,Italian Restaurant,Salon / Barbershop,Ice Cream Shop,Chinese Restaurant
33,Midtown South,Korean Restaurant,Coffee Shop,Hotel Bar,Boutique,Bakery,Hotel,Cosmetics Shop,Cocktail Bar,Japanese Restaurant,Gym / Fitness Center
37,Stuyvesant Town,Bar,Park,Heliport,Farmers Market,Basketball Court,Baseball Field,Beer Garden,Cocktail Bar,Coffee Shop,Harbor / Marina


## Analysis of Brooklyn neighborhoods, for reference only.

In [55]:
brooklyn_restaurants.sort_values(by='Restaurant count',ascending=False)

,Neighborhood,Restaurant count,Asian Restaurant count
56,South Side,36,2
14,Clinton Hill,34,0
1,Bay Ridge,31,0
44,North Side,28,1
15,Cobble Hill,26,0
28,Fort Greene,26,0
12,Carroll Gardens,25,0
8,Brooklyn Heights,24,2
20,Downtown,24,3
49,Prospect Heights,23,0


In [56]:
# one hot encoding
brooklyn_onehot = pd.get_dummies(brooklyn_venues[['Venue Category']], prefix="", prefix_sep="")
print(brooklyn_onehot.shape)
# add neighborhood column back to dataframe
brooklyn_onehot['Neighbourhood'] = brooklyn_venues['Neighborhood'] 
print(brooklyn_onehot.shape)
# move neighborhood column to the first column
fixed_columns = [brooklyn_onehot.columns[-1]] + list(brooklyn_onehot.columns[:-1])
brooklyn_onehot = brooklyn_onehot[fixed_columns]
#brooklyn_onehot.rename(columns={'Neighbourhood':'Neighborhood'},inplace=True)
brooklyn_onehot.head()


(2816, 286)
(2816, 287)


,Neighbourhood,Adult Boutique,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Studio,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Herbs & Spices Store,History Museum,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Non-Profit,Noodle House,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Piercing Parlor,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Print Shop,Pub,Public Art,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Club,Roller Rink,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Restaurant,Tiki Bar,Toy / Game Store,Trail,Turkish Restaurant,Used Bookstore,Vape Store,Varenyky restaurant,Vegetarian / Vegan Restaurant,Video 

In [57]:
brooklyn_grouped = brooklyn_onehot.groupby('Neighbourhood').mean().reset_index()
brooklyn_grouped

,Neighbourhood,Adult Boutique,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Studio,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Herbs & Spices Store,History Museum,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Non-Profit,Noodle House,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Piercing Parlor,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Print Shop,Pub,Public Art,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Club,Roller Rink,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Restaurant,Tiki Bar,Toy / Game Store,Trail,Turkish Restaurant,Used Bookstore,Vape Store,Varenyky restaurant,Vegetarian / Vegan Restaurant,Video 

In [58]:
num_top_venues = 5

for hood in brooklyn_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = brooklyn_grouped[brooklyn_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bath Beach----
                  venue  freq
0           Pizza Place  0.06
1            Donut Shop  0.06
2              Pharmacy  0.06
3  Fast Food Restaurant  0.04
4    Chinese Restaurant  0.04


----Bay Ridge----
                venue  freq
0  Italian Restaurant  0.09
1                 Spa  0.07
2         Pizza Place  0.05
3    Greek Restaurant  0.04
4          Bagel Shop  0.04


----Bedford Stuyvesant----
                     venue  freq
0              Coffee Shop  0.11
1                     Café  0.07
2              Pizza Place  0.07
3                      Bar  0.07
4  New American Restaurant  0.04


----Bensonhurst----
                venue  freq
0  Chinese Restaurant  0.12
1    Sushi Restaurant  0.06
2              Bakery  0.06
3      Ice Cream Shop  0.06
4   Convenience Store  0.03


----Bergen Beach----
                venue  freq
0     Harbor / Marina  0.33
1  Athletics & Sports  0.17
2          Playground  0.17
3          Donut Shop  0.17
4      Baseball Field  0.17


---

In [59]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
brooklyn_neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
brooklyn_neighborhoods_venues_sorted['Neighborhood'] = brooklyn_grouped['Neighbourhood']

for ind in np.arange(brooklyn_grouped.shape[0]):
    brooklyn_neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(brooklyn_grouped.iloc[ind, :], num_top_venues)

brooklyn_neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bath Beach,Pizza Place,Pharmacy,Donut Shop,Italian Restaurant,Sushi Restaurant,Kids Store,Fast Food Restaurant,Chinese Restaurant,Sandwich Place,Burger Joint
1,Bay Ridge,Italian Restaurant,Spa,Pizza Place,Greek Restaurant,Bagel Shop,American Restaurant,Bar,Playground,Cosmetics Shop,Pharmacy
2,Bedford Stuyvesant,Coffee Shop,Café,Pizza Place,Bar,BBQ Joint,New American Restaurant,Boutique,Gourmet Shop,Grocery Store,Cocktail Bar
3,Bensonhurst,Chinese Restaurant,Ice Cream Shop,Bakery,Sushi Restaurant,Dessert Shop,Smoke Shop,Butcher,Shabu-Shabu Restaurant,Russian Restaurant,Road
4,Bergen Beach,Harbor / Marina,Baseball Field,Athletics & Sports,Donut Shop,Playground,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop
5,Boerum Hill,Coffee Shop,Bar,Dance Studio,Cosmetics Shop,Kids Store,Grocery Store,Martial Arts Dojo,Sandwich Place,Furniture / Home Store,Chinese Restaurant
6,Borough Park,Pizza Place,Deli / Bodega,Bank,Pharmacy,Farmers Market,Restaurant,Candy Store,Metro Station,Men's Store,Bike Shop
7,Brighton Beach,Russian Restaurant,Beach,Restaurant,Eastern European Restaurant,Pharmacy,Gourmet Shop,Bank,Mobile Phone Shop,Neighborhood,Fast Food Restaurant
8,Broadway Junction,Diner,Donut Shop,Liquor Store,Playground,Sandwich Place,Caribbean Restaurant,Fried Chicken Joint,Business Service,Bus Stop,Bus Station
9,Brooklyn Heights,Yoga Studio,Park,Coffee Shop,Italian Restaurant,Deli / Bodega,Pizza Place,Cosmetics Shop,Gym,Bar,Juice Bar


In [60]:
# set number of clusters
brooklyn_kclusters =8

brooklyn_grouped_clustering = brooklyn_grouped.drop('Neighbourhood', 1)

# run k-means clustering
brooklyn_kmeans = KMeans(n_clusters=brooklyn_kclusters, random_state=0).fit(brooklyn_grouped_clustering)

# check cluster labels generated for each row in the dataframe
brooklyn_kmeans.labels_[0:10] 

array([1, 1, 1, 1, 7, 1, 3, 1, 1, 1], dtype=int32)

In [62]:
brooklyn_merged = brooklyn_data

# add clustering labels
brooklyn_merged['Cluster Labels'] = brooklyn_kmeans.labels_

brooklyn_merged = brooklyn_merged.join(brooklyn_neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

brooklyn_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Brooklyn,Bay Ridge,40.625801,-74.030621,1,Italian Restaurant,Spa,Pizza Place,Greek Restaurant,Bagel Shop,American Restaurant,Bar,Playground,Cosmetics Shop,Pharmacy
1,Brooklyn,Bensonhurst,40.611009,-73.995180,1,Chinese Restaurant,Ice Cream Shop,Bakery,Sushi Restaurant,Dessert Shop,Smoke Shop,Butcher,Shabu-Shabu Restaurant,Russian Restaurant,Road
2,Brooklyn,Sunset Park,40.645103,-74.010316,1,Pizza Place,Mexican Restaurant,Latin American Restaurant,Bank,Bakery,Gym,Pharmacy,Mobile Phone Shop,Deli / Bodega,Record Shop
3,Brooklyn,Greenpoint,40.730201,-73.954241,1,Bar,Pizza Place,Boutique,Cocktail Bar,Yoga Studio,Record Shop,Café,Coffee Shop,Mexican Restaurant,Bakery
4,Brooklyn,Gravesend,40.595260,-73.973471,7,Pizza Place,Italian Restaurant,Lounge,Deli / Bodega,Chinese Restaurant,Metro Station,Martial Arts Dojo,Spa,Record Shop,Moving Target
